In [1]:
import pandas as pd
import re
from dateutil.parser import parse
from datetime import datetime
from tqdm import tqdm
from glob import glob
from methods import *

In [2]:
data = pd.read_csv('scraping/comments.csv',encoding='utf-8')

In [3]:
data

,comment
0,اليمين المتطرف دار خير فالمغاربة ديال مليلية و...
1,للاسف نحن لازيد من 500سنة ولم نقدر على استرجاع...
2,اليمين المتطرف في مليلية يرفض ترسيم الأمازيغية...
3,رغم أنني أمازيغي أبا عن جد لا أجد جدوى من تدري...
4,الاسبان لم يتنازلوا عن عقليتهم الاستعمارية الم...
...,...
18328,المغرب متخلف جدا في نظامه البنكي والمالي. منذ ...
18329,للاسف البنوك المغربية مازالت متخلفة في طريقة ا...
18330,السلام عليكم ورحمة الله ،لماذا هذا التشائم دائ...
18331,كنصيفت مرات من الحساب ديالي من امريكا إلى حساب...


In [6]:
data['content'] = data['comment'].apply(clean)


In [7]:
data['content'] = data['content'].apply(remove_stopWords)

In [8]:
df = data

In [9]:
df.tail()

,comment,content
18328,المغرب متخلف جدا في نظامه البنكي والمالي. منذ ...,المغرب متخلف نظامه البنكي والمالي سنة 2014 الا...
18329,للاسف البنوك المغربية مازالت متخلفة في طريقة ا...,للاسف البنوك المغربية مازالت متخلفة طريقة التع...
18330,السلام عليكم ورحمة الله ،لماذا هذا التشائم دائ...,السلام ورحمة الله لماذا التشائم دائما أنجزه ال...
18331,كنصيفت مرات من الحساب ديالي من امريكا إلى حساب...,كنصيفت مرات الحساب ديالي امريكا حسابي فالمغرب ...
18332,إلى صاحب التعليق رقم 15:شكرا على تعليقك.\nكثير...,صاحب التعليق رقم 15 شكرا تعليقك كثير التعاليق ...


In [10]:
docs = df.content.to_list()
#months = df.month.to_list()
# docs

In [11]:
# docs = docs[:10000]
len(docs)

18333

# Topic Modeling

In [12]:
from bertopic import BERTopic

In [11]:
# from sentence_transformers import SentenceTransformer
# from umap import UMAP
# from hdbscan import HDBSCAN

# embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
# umap_model = UMAP(n_neighbors=3, n_components=3, min_dist=0.05)
# hdbscan_model = HDBSCAN(min_cluster_size=80, min_samples=40,
#                         gen_min_span_tree=True,
#                         prediction_data=True)

In [12]:
# topic_model = BERTopic(
#     umap_model=umap_model,
#     hdbscan_model=hdbscan_model,
#     embedding_model=embedding_model,
#     vectorizer_model=vectorizer_model,
#     top_n_words=5,
#     language='english', #multilingual
#     calculate_probabilities=True,
#     verbose=True
# )

In [13]:
topic_model = BERTopic(language="arabic",
                        calculate_probabilities=False,
                        verbose=True,
                        min_topic_size=75)

### Training

We start by instantiating BERTopic. We set language to `english` since our documents are in the English language. If you would like to use a multi-lingual model, please use `language="multilingual"` instead. 

We will also calculate the topic probabilities. However, this can slow down BERTopic significantly at large amounts of data (>100_000 documents). It is advised to turn this off if you want to speed up the model. 

In [14]:
topics, probs = topic_model.fit_transform(docs)

Batches:   0%|          | 0/573 [00:00<?, ?it/s]

2023-05-03 10:51:13,972 - BERTopic - Transformed documents to Embeddings
2023-05-03 10:52:25,858 - BERTopic - Reduced dimensionality
2023-05-03 10:52:34,239 - BERTopic - Clustered reduced embeddings


### Extracting Topics

In [15]:
freq = topic_model.get_topic_info()
freq

,Topic,Count,Name
0,-1,6506,-1_الحكومة_المغرب_ان_الله
1,0,2015,0_الله_او_اللهم_ان
2,1,1968,1_الأمازيغية_الامازيغية_اللغة_لغة
3,2,1212,2_الحكومة_الدولة_الشعب_ان
4,3,673,3_موطني_انا_شيء_او
5,4,654,4_المغرب_المغاربة_المغربية_المغربي
6,5,515,5_الله_بالله_الوكيل_قوة
7,6,380,6_تخدمو_مانو_وبشارة_الانتخابي
8,7,356,7_الامازيغية_الأمازيغية_اللغة_العربية
9,8,288,8_التعليم_إصلاح_التعليمية_الأساتذة


In [18]:
topic_model.get_topic(12
                      )  # Select the most frequent topic

[('الجزائر', 0.09755286167810616),
 ('المغرب', 0.03741185602842759),
 ('الجزائري', 0.030382781598224815),
 ('دولة', 0.02137510115072494),
 ('الصحراء', 0.02137510115072494),
 ('الحدود', 0.020038081172001364),
 ('النظام', 0.01863448748692907),
 ('ان', 0.01464787380800867),
 ('الكبرانات', 0.013517885010149815),
 ('الشرقية', 0.013171950778846721)]

In [11]:
topic_model.topics_[:10]

[61, 81, -1, -1, 2, 34, -1, -1, -1, 2]

# Visualization
There are several visualization options available in BERTopic, namely the visualization of topics, probabilities and topics over time. Topic modeling is, to a certain extent, quite subjective. Visualizations help understand the topics that were created.

### Visualize Topics

After having trained our BERTopic model, we can iteratively go through perhaps a hundred topic to get a good understanding of the topics that were extract. However, that takes quite some time and lacks a global representation. Instead, we can visualize the topics that were generated in a way very similar to LDAvis:

In [19]:
topic_model.visualize_topics(top_n_topics=75)

### Visualize Topic Probabilities

The variable `probabilities` that is returned from `transform()` or `fit_transform()` can 
be used to understand how confident BERTopic is that certain topics can be found in a document. 

To visualize the distributions, we simply call:

In [15]:
# topic_model.visualize_distribution(probs[200], min_probability=0.015)

### Visualize Topic Hierarchy

The topics that were created can be hierarchically reduced. In order to understand the potential hierarchical structure of the topics, we can use scipy.cluster.hierarchy to create clusters and visualize how they relate to one another. This might help selecting an appropriate nr_topics when reducing the number of topics that you have created.

In [20]:
topic_model.visualize_hierarchy(top_n_topics=75)

### Visualize Terms

We can visualize the selected terms for a few topics by creating bar charts out of the c-TF-IDF scores for each topic representation. Insights can be gained from the relative c-TF-IDF scores between and within topics. Moreover, you can easily compare topic representations to each other.

In [21]:
topic_model.visualize_barchart(top_n_topics=10)

In [22]:
topic_model.get_representative_docs()

{-1: ['انقسام مجلس الأمن محاكمة مرتكبي الجرائم دارفور انقسم مجلس الامن التابع للامم المتحدة يوم امس الاول بشأن المكان ينبغي ان يحاكم مرتكبو جرائم الحرب منطقة دارفور بالسودان اتخاذ اوروبا والصين والولايات المتحدة مواقف مختلفة يرى الدبلوماسيون حلا سهلا للمشكلة غضون اتفقت الاطراف المتحاربة منطقة دارفور السودانية امس الاول استئناف محادثات السلام المتعثرة ان دعا الامين العام للأمم المتحدة كوفي عنان الى اتخاذ خطوات صارمة لوضع نهاية للصراع وصفه بانه يوشك ان يكون جحيما الارض وللمرة الاولى قال اعضاء مجلس الأمن الخمسة عشر انهم يحبذون احالة المشتبه ارتكابهم جرائم حرب منطقة دارفور السودانية الى المحكمة الجنائية الدولية لاهاي ترفضه بشدة الولايات المتحدة يجر تصويت رسمي المسألة وقال دبلوماسيون ان المعارضة المشاورات جرت متاخر يوم الاربعاء جاءت الصين والجزائر جادلتا بان محاولة مقاضاة زعماء الحكومة السوادنية او المتمردين او ميليشيات الجنجويد احلال السلام دارفور يأتي بنتائج عكسية واتفقتا الحكومة السودانية ردها القائل بان المقاضاة جرائم خطيرة حق البشرية يجب ان يكون امام محاكم سودانية وقالت الصين والجزائر 

## Predict

We can predict what topic any arbitrary text belongs to using the fitted model. We can accomplish this by calling the transform() method. The code below demonstrates this and uses a made up headline for a news article

In [86]:
# text = "ياله خديت التيكي ديالي باش نحضر بلابلاكونف اللي منضمة من طرف الكومينوتي غيغس بلابلا. اش مازال كتسنى، بالي قبل ما يسالي و قطع ورقتك حتى نتا"
text = 'One of the sessions that I LOVED today ! Design thinking enables us to create Solutions that are Desirable, Feasible and Viable !'

preds, probs = topic_model.transform(text)
print(preds)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2022-12-20 00:45:31,887 - BERTopic - Reduced dimensionality
2022-12-20 00:45:31,906 - BERTopic - Predicted clusters


[-1]


In [87]:
topic_model.get_topic(-1)

[('in the', 0.0034637327245780353),
 ('of the', 0.0034136140857753423),
 ('is the', 0.0024549142647816275),
 ('you can', 0.0023382883893685474),
 ('this is', 0.002324523104440822),
 ('if you', 0.0021950792753658572),
 ('for the', 0.0021695064518143783),
 ('on the', 0.0021192969416989286),
 ('to the', 0.0021112693398216936),
 ('to be', 0.0020038755018632025)]

## Similarity Check

In [22]:
topic_model.visualize_heatmap(topics=[9], top_n_topics=5)

In [23]:
similar_topics, similarity = topic_model.find_topics("java", top_n=5)

for similar_topic in similar_topics:
    print(similar_topic)
    print(topic_model.get_topic(similar_topic))

0
[('الله', 0.022901768291296847), ('او', 0.012810800118076716), ('اللهم', 0.012624582358286611), ('ان', 0.010928434591189942), ('والله', 0.008690046469479231), ('اللي', 0.008415438847043044), ('الى', 0.007368369843283555), ('راه', 0.00695551456572512), ('يا', 0.006895139774023939), ('ديال', 0.006645064389355512)]
3
[('موطني', 0.013366618546520432), ('انا', 0.012526694074509728), ('شيء', 0.012421726967279582), ('او', 0.011256684970809349), ('واحد', 0.010233981915920981), ('واش', 0.009919498347003456), ('كلام', 0.009688542954846474), ('زيد', 0.009565013172659575), ('هاد', 0.009069034977215184), ('شي', 0.008628364842035247)]
6
[('تخدمو', 0.3144017675942338), ('مانو', 0.3070720281044521), ('وبشارة', 0.3027472166955851), ('الانتخابي', 0.30072415699896704), ('تترجم', 0.2969197758975618), ('وهدشي', 0.29339800016812695), ('برنامجها', 0.291730572196365), ('الأمريكية', 0.2814593369354547), ('خصكم', 0.2764565082910119), ('الجواب', 0.26400596230385587)]
21
[('السوسية', 0.14695903348329983), ('الن

In [24]:
similar_topics, similarity = topic_model.find_topics("maroc", top_n=5)
topic_model.get_topic(similar_topics[0])

[('المغرب', 0.021125561824419718),
 ('المغاربة', 0.019337369977733648),
 ('المغربية', 0.015663331258518962),
 ('المغربي', 0.013919954729239131),
 ('الله', 0.013658376582902839),
 ('ان', 0.01279106109579365),
 ('الى', 0.011086627574428442),
 ('الشعب', 0.00883937599580444),
 ('التاريخ', 0.008171882197830703),
 ('او', 0.008152402637401775)]

# Topics over Time

Although extracting the topics and their representation is interesting, we are missing some dimensional information. For example, some topics might not be relevant anymore or some are gaining traction over the last years. That can be vital information when making decisions. 

Here, we will model the topics over the years. For each topic and timestep, we calculate the c-TF-IDF representation. This will result in a specific topic representation at each timestep without the need to create clusters from embeddings as they were already created. However, topics can be regarded as evolutionary entities that evolve and change over time. This means that a topic representation at timestep *t* should be related to its representation at timesetps *t-1* and *t+1*. To model this evolutionary trend, we average its c-TF-IDF representation with that of the c-TF-IDF representation at timestep *t-1*. This is done for each topic representation allowing for the representations to evolve over time.

In [49]:
topics_over_time = topic_model.topics_over_time(docs, months)

12it [00:17,  1.49s/it]


In [25]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=75, width=1000, height=700)

NameError: name 'topics_over_time' is not defined

In [26]:
topic_model.visualize_topics_over_time(topics_over_time, width=1000, height=700,
                                        topics=[9, 25, 53, 63, 66, 74])

NameError: name 'topics_over_time' is not defined

### Visualize Topic Similarity

Having generated topic embeddings, through both c-TF-IDF and embeddings, we can create a similarity matrix by simply applying cosine similarities through those topic embeddings. The result will be a matrix indicating how similar certain topics are to each other.

In [57]:
topic_model.visualize_heatmap(n_clusters=5, width=1000, height=1000)

### Visualize Term Score Decline

Topics are represented by a number of words starting with the best representative word. Each word is represented by a c-TF-IDF score. The higher the score, the more representative a word to the topic is. Since the topic words are sorted by their c-TF-IDF score, the scores slowly decline with each word that is added. At some point adding words to the topic representation only marginally increases the total c-TF-IDF score and would not be beneficial for its representation.

To visualize this effect, we can plot the c-TF-IDF scores for each topic by the term rank of each word. In other words, the position of the words (term rank), where the words with the highest c-TF-IDF score will have a rank of 1, will be put on the x-axis. Whereas the y-axis will be populated by the c-TF-IDF scores. The result is a visualization that shows you the decline of c-TF-IDF score when adding words to the topic representation. It allows you, using the elbow method, the select the best number of words in a topic.

In [27]:
topic_model.visualize_term_rank()

# Topic Representation

After having created the topic model, you might not be satisfied with some of the parameters you have chosen. Fortunately, BERTopic allows you to update the topics after they have been created.

This allows for fine-tuning the model to your specifications and wishes.

### Update Topics

When you have trained a model and viewed the topics and the words that represent them,
you might not be satisfied with the representation. Perhaps you forgot to remove
stopwords or you want to try out a different `n_gram_range`. We can use the function `update_topics` to update 
the topic representation with new parameters for `c-TF-IDF`: 

In [28]:
topic_model_updated = topic_model
topic_model_updated.update_topics(docs, n_gram_range=(2, 3))

In [29]:
topic_model_updated.get_topic(0)   # We select topic that we viewed before

[('الله الوكيل', 0.0021337346804413933),
 ('الحمد لله', 0.001781233800843006),
 ('صلى الله', 0.0017213739525889678),
 ('اللغة العربية', 0.0015176615098822844),
 ('الله وسلم', 0.0015079866086657413),
 ('صلى الله وسلم', 0.0014402323312345277),
 ('محمد السادس', 0.0011139887806020307),
 ('رحمه الله', 0.001099757917595821),
 ('ياخد الحق', 0.0010808627144163285),
 ('الا بالله', 0.0010594807326497438)]

### Topic Reduction

We can also reduce the number of topics after having trained a BERTopic model. The advantage of doing so, is that you can decide the number of topics after knowing how many are actually created. It is difficult to predict before training your model how many topics that are in your documents and how many will be extracted. Instead, we can decide afterwards how many topics seems realistic:

In [30]:
topic_model_updated.reduce_topics(docs, nr_topics=20)

2023-05-03 11:08:28,007 - BERTopic - Reduced number of topics from 35 to 20


In [25]:
# Access the newly updated topics with:
# print(topic_model.topics_)

# Search Topics

After having trained our model, we can use `find_topics` to search for topics that are similar 
to an input search_term. Here, we are going to be searching for topics that closely relate the 
search term "vehicle". Then, we extract the most similar topic and check the results: 

In [32]:
similar_topics, similarity = topic_model_updated.find_topics("الشعب", top_n=5); similar_topics

[15, -1, 3, 14, 5]

In [33]:
topic_model.get_topic(15)

[('التجمع الوطني', 0.08855402197330207),
 ('حزب التجمع', 0.07789391977616394),
 ('حزب التجمع الوطني', 0.07634419020673462),
 ('التجمع الوطني للأحرار', 0.05798428835309957),
 ('الوطني للأحرار', 0.05798428835309957),
 ('الوطني للاحرار', 0.035799699147669964),
 ('التجمع الوطني للاحرار', 0.035799699147669964),
 ('حزب الحمامة', 0.03530619015657253),
 ('الأحزاب السياسية', 0.021280736599493005),
 ('حزب الاحرار', 0.01741794450632275)]

# Model serialization
The model and its internal settings can easily be saved. Note that the documents and embeddings will not be saved. However, UMAP and HDBSCAN will be saved.

In [34]:
# Save model
topic_model.save("my_model_all")	

c:\Users\Mustapha\anaconda3\envs\renv\lib\site-packages\scipy\sparse\_index.py:146: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [35]:
# Load model
my_model = BERTopic.load("my_model_all")

# Embedding Models

The parameter `embedding_model` takes in a string pointing to a sentence-transformers model, a SentenceTransformer, or a Flair DocumentEmbedding model.

### Sentence-Transformers

You can select any model from sentence-transformers here and pass it through BERTopic with embedding_model:


In [39]:
topic_model = BERTopic(embedding_model="xlm-r-bert-base-nli-stsb-mean-tokens")

Or select a SentenceTransformer model with your own parameters:

In [ ]:
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer("distilbert-base-nli-mean-tokens", device="cpu")
topic_model = BERTopic(embedding_model=sentence_model, verbose=True)